# Reconocimiento facial: python + tensorflow + keras

In [84]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import os
import pickle
import time
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from os import getenv
from dotenv import load_dotenv  # pipenv install python-dotenv

load_dotenv()

FACENET_KERAS_WEIGHTS = getenv("FACENET_KERAS_WEIGHTS")
LABEL_ENCODER = getenv("LABEL_ENCODER")
DATASET_DIR = getenv("DATASET_DIR")
FACENET_PREDICTOR = getenv("FACENET_PREDICTOR")

In [ ]:
tf.__version__

'2.7.0'

In [23]:
def compute_embedding(model, face):
    face = face.astype('float32')
    
    mean, std = face.mean(), face.std()
    face = (face-mean) / std
    
    face = np.expand_dims(face,axis=0)
    
    embedding = model.predict(face)
    return embedding

In [4]:
# Create the FaceNet model
from architecture import InceptionResNetV2

face_encoder = InceptionResNetV2()

# # Load the weights of the model
face_encoder.load_weights(FACENET_KERAS_WEIGHTS)

## Múltiples caras

In [ ]:
model_label_encoder = pickle.load(open(LABEL_ENCODER, "rb"))
ids2name = {idx: name for idx, name in enumerate(model_label_encoder.classes_)}
name2ids = {name: idx for idx, name in enumerate(model_label_encoder.classes_)}

X = []
y = []
for idx, files in enumerate(os.walk(DATASET_DIR)):
    if idx > 0:
        person_dir = files[0]
        for file in files[2]:
            image = cv2.cvtColor(cv2.imread(os.path.join(person_dir, file)), cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, dsize=(160, 160))
            embedding = compute_embedding(face_encoder, image)
            X.append(embedding[0])
            personId = name2ids[person_dir.split('/')[2]]
            y.append(personId)

In [ ]:
X_train , X_test , Y_train , Y_test = train_test_split(X, y, test_size=0.3, random_state=0)

t_init=time.time()
svm = SVC(kernel='rbf' , class_weight='balanced' , C=1000 , gamma=0.0082)
svm.fit(X_train , Y_train)
print(svm.score(X_test , Y_test))
print("Hecho en %0.3fs" % (time.time() - t_init))

0.9803921568627451
Hecho en 0.005s


In [ ]:
pickle.dump(svm, open(FACENET_PREDICTOR, 'wb'))